# 1. Expected Goal Model using logistic regression

In [4]:
import sys
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import math

project_root = os.path.abspath("..")

if project_root not in sys.path:
    sys.path.append(project_root)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as sk_metrics
from sklearn.metrics import precision_score

import pyodbc
from sqlalchemy import create_engine
from preprocessing.event_data import load_and_process_event_data


In [5]:
df_events = load_and_process_event_data()

SystemError: <class 'pyodbc.Error'> returned a result with an exception set

In [ ]:
path = "/home/datgegt/projects/tf216/football-data-visualization-and-analysis-dashboard/dataset/event_data.csv"
df_events.to_csv(path)

In [ ]:
df_events.head()

,matchID,matchPeriod,eventSec,eventName,teamID,playerID,subEventName,Description,posOrigXMeters,posDestXMeters,posOrigYMeters,posDestYMeters
0,2499719,1H,2.758649,Pass,1609,25413.0,Simple pass,Accurate,51.45,32.55,33.32,53.04
1,2499719,1H,4.946850,Pass,1609,370224.0,High pass,Accurate,32.55,53.55,53.04,51.00
2,2499719,1H,6.542188,Pass,1609,3319.0,Head pass,Accurate,53.55,36.75,51.00,48.28
3,2499719,1H,8.143395,Pass,1609,120339.0,Head pass,Accurate,36.75,43.05,48.28,64.60
4,2499719,1H,10.302366,Pass,1609,167145.0,Simple pass,Accurate,43.05,75.60,64.60,59.84


In [ ]:
df_shots = df_events[df_events['eventName'] == 'Shot'].copy()

In [ ]:
df_shots

,matchID,matchPeriod,eventSec,eventName,teamID,playerID,subEventName,Description,posOrigXMeters,posDestXMeters,posOrigYMeters,posDestYMeters
80,2499719,1H,94.595788,Shot,1609,25413.0,Shot,Goal,92.40,0.0,27.88,0.00
81,2499719,1H,94.595788,Shot,1609,25413.0,Shot,Opportunity,92.40,0.0,27.88,0.00
82,2499719,1H,94.595788,Shot,1609,25413.0,Shot,Right foot,92.40,0.0,27.88,0.00
83,2499719,1H,94.595788,Shot,1609,25413.0,Shot,Position: Goal low left,92.40,0.0,27.88,0.00
84,2499719,1H,94.595788,Shot,1609,25413.0,Shot,Accurate,92.40,0.0,27.88,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4909378,2058017,2H,2487.443304,Shot,4418,28115.0,Shot,Accurate,5.25,16.8,42.84,42.84
4909416,2058017,2H,2589.084194,Shot,9598,3476.0,Shot,Opportunity,66.15,63.0,51.00,61.20
4909417,2058017,2H,2589.084194,Shot,9598,3476.0,Shot,Right foot,66.15,63.0,51.00,61.20
4909418,2058017,2H,2589.084194,Shot,9598,3476.0,Shot,Position: Out high right,66.15,63.0,51.00,61.20


In [ ]:
total_shots = len(df_shots)
print(f"Tổng số cú sút: {total_shots}")
print(f"Xác suất ghi bàn khi thực hiện cú sút: {df_shots['Goal'].mean()*100:.2f}%")

Tổng số cú sút: 166939


KeyError: 'Goal'

## TRAIN MODEL WITH LOGISTIC REGRESSION

In [ ]:
feature_cols_ = ["posOrigXMeters", "posOrigYMeters", "bodyPartShotCode", "CounterAttack"]
tagets_ = ['Goal']

X = df_shots[feature_cols_]
y = df_shots[tagets_]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
regress_shitty = LogisticRegression(random_state=42, multi_class='auto')
regress_shitty.fit(X_train, np.array(y_train).ravel())

/home/datgegt/miniconda3/envs/tf216/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,100
,multi_class,'auto'


In [ ]:
predict_vals = regress_shitty.predict(X_test)

In [ ]:
print(f"Accuracy of the model is {sk_metrics.accuracy_score(y_test, predict_vals)*100:.1f}%")

Accuracy of the model is 89.5%


In [ ]:
for i, col in enumerate(X_train.columns):
    print(f"Coefficent of {col}: {regress_shitty.coef_[0][i]:.4f}")

Coefficent of posOrigXMeters: -0.0013
Coefficent of posOrigYMeters: -0.0004
Coefficent of bodyPartShotCode: -0.0375
Coefficent of CounterAttack: 0.4512


<b>posOrigXMeters</b>: Với mỗi vị trí sút theo trục X tăng thêm 1 mét, giá trị dự đoán của mô hình giảm trung bình đi 0.13%. Điều này phản anh rằng các cầu thủ càng xa khung thành theo trục X, khả năng thành công càng giảm.  
<b>posOrigYMeters</b>: khi vị trí cầu thủ sút lệch sang trái/phải thêm 1 mét, giá trị dự đoán giảm 0.04%.mức ảnh hưởng nhỏ hơn so với trục X.  
<b>bodyPartShotCode</b>: Bộ phận cơ thể có giá trị âm điều này phản ánh rằng bộ phận cơ thể làm giảm khả năng ghi bàn.  
<b>CounterAttack</b>: Các pha phản công có xác suất sút cao hơn bằng cút sút nguy hiểm/ghi bàn cao hơn với xác suất được mô hình đánh giá là 45.12%.  


#### Log loss

In [ ]:
predict_probs = regress_shitty.predict_proba(X_test)[:, 1]
print(f"Log loss of our model: {sk_metrics.log_loss(y_test, predict_probs):.5f}")

Log loss of our model: 0.33510


Với *Log loss = 0.33510*, mô hinh cho thấy đạt mức độ dự đoán khá tốt so với bộ dữ liệu có tỉ lệ ghi bàn thấp *(10.42%)*. Giá trị này cho thấy mô hình đang tạo ra xác suất dự đoán tương đối chính xác và ổn định, mặc dù vẫn chưa vượt nhiều mong đợi so với baseline xác suất trung bình.

#### AUC

In [ ]:
print(f"AUC of our model: {sk_metrics.roc_auc_score(y_test, predict_probs)*100:.2f}%")

AUC of our model: 53.83%


In [ ]:
precisions = precision_score(y_train, y_test)
print(f"F1-Score of our model: {sk_metrics.f1_score(y_test, predict_probs)*100:.2f}%")

ValueError: Classification metrics can't handle a mix of binary and continuous targets

cols= [eventName, teamID, ]